# Pixeltable MCP with CrewAI Flow

This notebook demonstrates a sophisticated multimodal AI system that uses CrewAI Flows to intelligently route and process different types of media queries. The system includes specialized agents for handling images, videos, audio, and documents using Pixeltable's indexing and search capabilities.

## Overview

The **PixeltableManagerFlow** leverages:
- **CrewAI Flows**: For orchestrating multi-agent workflows
- **Pixeltable Tools**: For multimodal data indexing and retrieval
- **Specialized Agents**: Each expert in their respective domain (image, video, audio, document)
- **Intelligent Routing**: Automatic query classification and delegation

## 📦 Required Imports

Import all necessary libraries for the implementation:

In [ ]:
import os
from typing import Dict, Any
from IPython.display import display, Markdown, HTML
from pydantic import BaseModel
from crewai import Agent, Task, Crew
from crewai_tools import MCPServerAdapter
from crewai import LLM
from crewai.flow.flow import Flow, router, start, listen, or_

## 🤖 LLM Configuration

Configure the Language Models for different purposes:
- **Agent LLM**: Used by specialist agents for executing tasks
- **Router LLM**: Used by the routing agent for query classification

In [ ]:
os.environ['OPENAI_API_KEY'] = 'YOUR-API-KEY'

In [ ]:
# Define our LLMs for providing to agents
agent_llm = LLM(model="openai/gpt-4")
router_llm = LLM(model="ollama/gemma3")

## 🌐 Pixeltable Server Configuration

Configure the MCP (Model Context Protocol) servers for different Pixeltable services:
- **Image Server** (Port 8080): Handles image indexing and search
- **Video Server** (Port 8081): Handles video transcription and search  
- **Audio Server** (Port 8082): Handles audio transcription and search
- **Document Server** (Port 8083): Handles document indexing and search

In [ ]:
# Pixeltable SSE Servers
server_configurations = [
    {
        "url": "https://8080-01jyezrk9g2fhjb5156dcj7seq.cloudspaces.litng.ai/sse",
        "transport": "sse"
    },
    {
        "url": "https://8081-01jyezrk9g2fhjb5156dcj7seq.cloudspaces.litng.ai/sse",
        "transport": "sse"
    },
    {
        "url": "https://8082-01jyezrk9g2fhjb5156dcj7seq.cloudspaces.litng.ai/sse",
        "transport": "sse"
    },
    {
        "url": "https://8083-01jyezrk9g2fhjb5156dcj7seq.cloudspaces.litng.ai/sse",
        "transport": "sse"
    },
]

## 🔄 PixeltableManagerFlow Class

The main flow class that orchestrates the entire multimodal AI system. This implementation includes:

### Key Components:
1. **State Management**: Using Pydantic models for type-safe state handling
2. **Intelligent Routing**: Automatic query classification to determine the appropriate specialist
3. **Specialist Agents**: Dedicated agents for each media type
4. **Result Synthesis**: Final processing to create user-friendly responses

### Flow Architecture:
```
User Query → Router Agent → Specialist Agent → Synthesis Agent → Final Result
```

### Supported Operations:
- **Image**: Index creation, image insertion, semantic search
- **Video**: Index creation, video insertion, transcription-based search
- **Audio**: Index creation, audio insertion, speech-to-text search
- **Document**: Index creation, document insertion, text-based search

In [ ]:
class PixeltableFlowState(BaseModel):
    query: str = ""
    result: str = ""


class Specialist(BaseModel):
    name: str


class PixeltableManagerFlow(Flow[PixeltableFlowState]):
    """
    A CrewAI Flow to intelligently delegate multimodal indexing/search queries
    to specialized agents using Pixeltable tools.
    """

    @start()
    def start_flow(self) -> Dict[str, Any]:
        print(f"Flow started with query: {self.state.query}")
        return {"query": self.state.query}

    @router(start_flow)
    def route_query(self) -> str:
        """Uses an agent to determine the correct specialist for the user's query."""
        print("--- Using Router Agent to classify query ---")
        router_agent = Agent(
            role="Task Router",
            goal=(
                "Analyze the user's query and determine the correct specialist. "
                "The available specialists are: 'image_specialist', 'video_specialist', "
                "'audio_specialist', 'document_specialist'."
            ),
            backstory="An AI expert in routing tasks to the appropriate specialist agent.",
            llm=router_llm,
        )

        routing_task = Task(
            description=f"Analyze the following user query and determine which specialist should handle it: '{self.state.query}'.",
            expected_output="""
Your output MUST be a single string from the following options:
- 'image_specialist'
- 'video_specialist'
- 'audio_specialist'
- 'document_specialist'
- 'default_specialist'
""",
            agent=router_agent,
            output_json=Specialist,
        )

        # Execute the routing task
        crew = Crew(agents=[router_agent], tasks=[routing_task], verbose=True)
        route = crew.kickoff()
        specialist = route["name"]
        return specialist

    @listen("image_specialist")
    def handle_image_task(self) -> Dict[str, Any]:
        print("--- Delegating to Image Specialist ---")
        try:
            with MCPServerAdapter(server_configurations) as mcp_tools:
                agent = Agent(
                    role="Image Specialist",
                    goal="Understand user queries and perform image indexing, insert images to index, and search operations using Pixeltable.",
                    backstory=(
                        "You are trained in visual data management. With expertise in semantic image embedding, "
                        "vector search retrieval, and index optimization, you transform image data into searchable visual knowledge bases."
                    ),
                    tools=[
                        mcp_tools["setup_image_index"],
                        mcp_tools["insert_image"],
                        mcp_tools["query_image"],
                        mcp_tools["list_image_tables"],
                    ],
                    llm=agent_llm,
                )

                task = Task(
                    description=f"""
Interpret the following user query and take the appropriate image-related actions using Pixeltable's tools: '{self.state.query}'.

You may:
- Create image index with table name
- Insert image file to specified image index
- Query specified image index by text question
- List existing image indexes

Choose the best actions to meet the user's request.
""",
                    expected_output="""
Output should include:
- Reasoning of performed operation
- Summary of successful tool use
- Recommended next actions

Format output as markdown.
""",
                    agent=agent,
                    markdown=True,
                )

                crew = Crew(agents=[agent], tasks=[task], verbose=True)
                result = crew.kickoff()
                return {"result": str(result)}

        except Exception as e:
            return {"result": f"Error processing image task: {e}"}

    @listen("video_specialist")
    def handle_video_task(self) -> Dict[str, Any]:
        print("--- Delegating to Video Specialist ---")
        try:
            with MCPServerAdapter(server_configurations) as mcp_tools:
                agent = Agent(
                    role="Video Specialist",
                    goal="Understand user queries and perform video indexing, insert videos to index, transcription, and search operations using Pixeltable.",
                    backstory=(
                        "You specialize in understanding visual and spoken information in videos. "
                        "Using transcription, indexing, and retrieval techniques, you make video data searchable and useful."
                    ),
                    tools=[
                        mcp_tools["setup_video_index"],
                        mcp_tools["insert_video"],
                        mcp_tools["query_video"],
                        mcp_tools["list_video_tables"],
                    ],
                    llm=agent_llm,
                )

                task = Task(
                    description=f"""
Interpret the following user query and take the appropriate video-related actions using Pixeltable's tools: '{self.state.query}'.

You may:
- Create video index with table name
- Insert video file to specified video index
- Query specified video index by text question
- List existing video indexes

Choose the best actions to meet the user's request.
""",
                    expected_output="""
Output should include:
- Reasoning of performed operation
- Summary of successful tool use
- Recommended next actions

Format output as markdown.
""",
                    agent=agent,
                    markdown=True,
                )

                crew = Crew(agents=[agent], tasks=[task], verbose=True)
                result = crew.kickoff()
                return {"result": str(result)}

        except Exception as e:
            return {"result": f"Error processing video task: {e}"}

    @listen("audio_specialist")
    def handle_audio_task(self) -> Dict[str, Any]:
        print("--- Delegating to Audio Specialist ---")
        try:
            with MCPServerAdapter(server_configurations) as mcp_tools:
                agent = Agent(
                    role="Audio Specialist",
                    goal="Understand user queries and perform audio indexing, adding audio content to index, transcription, and search operations using Pixeltable.",
                    backstory=(
                        "With advanced Whisper transcription models and semantic search tools, you specialize in turning "
                        "spoken content into searchable knowledge graphs."
                    ),
                    tools=[
                        mcp_tools["setup_audio_index"],
                        mcp_tools["insert_audio"],
                        mcp_tools["query_audio"],
                        mcp_tools["list_audio_tables"],
                    ],
                    llm=agent_llm,
                )

                task = Task(
                    description=f"""
Interpret the following user query and take the appropriate audio-related actions using Pixeltable's tools: '{self.state.query}'.

You may:
- Create audio indexe with table name
- Insert audio file to specified audio index
- Query specified audio index by text question
- List existing audio indexes

Choose the best actions to meet the user's request.
""",
                    expected_output="""
Output should include:
- Reasoning of performed operation
- Summary of successful tool use
- Recommended next actions

Format output as markdown.
""",
                    agent=agent,
                    markdown=True,
                )

                crew = Crew(agents=[agent], tasks=[task], verbose=True)
                result = crew.kickoff()
                return {"result": str(result)}

        except Exception as e:
            return {"result": f"Error processing audio task: {e}"}

    @listen("document_specialist")
    def handle_document_task(self) -> Dict[str, Any]:
        print("--- Delegating to Document Specialist ---")
        try:
            with MCPServerAdapter(server_configurations) as mcp_tools:
                agent = Agent(
                    role="Document Specialist",
                    goal="Understand user queries and perform document indexing, insertion, and search operations using Pixeltable.",
                    backstory=(
                        "You are an expert in document parsing and information retrieval. From PDFs to large text document corpus, "
                        "you chunk, index, and semantically search textual data with precision."
                    ),
                    tools=[
                        mcp_tools["setup_document_index"],
                        mcp_tools["insert_document"],
                        mcp_tools["query_document"],
                        mcp_tools["list_document_tables"],
                    ],
                    llm=agent_llm,
                )

                task = Task(
                    description=f"""
Interpret the following user query and take the appropriate document-related actions using Pixeltable's tools: '{self.state.query}'.

You may:
- Create document index with table name
- Insert document file to specified document index
- Query specified document index by text question
- List existing document indexes

Choose the best actions to meet the user's request.
""",
                    expected_output="""
Output should include:
- Reasoning of performed operation
- Summary of successful tool use
- Recommended next actions

Format output as markdown.
""",
                    agent=agent,
                    markdown=True,
                )

                crew = Crew(agents=[agent], tasks=[task], verbose=True)
                result = crew.kickoff()
                return {"result": str(result)}

        except Exception as e:
            return {"result": f"Error processing document task: {e}"}

    @listen("default_specialist")
    def handle_default_task(self) -> Dict[str, Any]:
        print("--- No specific specialist found, providing a general response. ---")
        return {
            "result": (
                "I'm not sure which type of content you're referring to.\n\n"
                "Please try rephrasing your query to mention one of the following types:\n"
                "- image\n- video\n- audio\n- document (PDF, text)"
            )
        }

    @listen(
        or_(
            "handle_image_task",
            "handle_video_task",
            "handle_audio_task",
            "handle_document_task",
            "handle_default_task",
        )
    )
    def synthesize_result(self, specialist_output: str):
        """Uses a synthesis agent to create a user-friendly final response."""
        print("--- Synthesizing Final Response ---")
        synthesis_agent = Agent(
            role="Synthesis Specialist",
            goal="Craft a clear, concise, and user-friendly response based on the technical output from the specialist agent.",
            backstory="An expert in communication and summarization.",
            llm=agent_llm,
        )
        synthesis_task = Task(
            description=f"Review the following technical result and present it as a final answer to the user. Final Result: '{specialist_output}'",
            expected_output="A polished, final response suitable for the end-user.",
            agent=synthesis_agent,
        )

        crew = Crew(agents=[synthesis_agent], tasks=[synthesis_task], verbose=True)
        final_result = crew.kickoff()
        return {"result": str(final_result)}

## 🚀 Usage Example

Demonstrate how to use the PixeltableManagerFlow with different types of queries:

### Example Queries:
1. **Video Index Creation**: `"Create a new video index with the table name as 'chatterbox_video'."`
2. **Video Insertion**: `"Add this video with URL as, 'https://...' to video index 'chatterbox_video'."`
3. **Video Search**: `"What was the discussed topic for the video stored in video index 'chatterbox_video'?"`

The flow will automatically:
- Classify the query type
- Route to the appropriate specialist
- Execute the required Pixeltable operations
- Synthesize a user-friendly response

Let's first preview our video before execution

In [3]:
video_url = "https://github.com/user-attachments/assets/12b341b2-e6ec-4bee-bf87-3109f726f91b"

HTML(f"""
<video width="640" height="480" controls>
  <source src="{video_url}" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

In [5]:
# Usage example
flow = PixeltableManagerFlow()
# result = await flow.kickoff_async(inputs={'query': "Create a new video index with the table name as 'chatterbox_video'."})
# result = await flow.kickoff_async(inputs={'query': "Add this video with URL as, 'https://github.com/user-attachments/assets/12b341b2-e6ec-4bee-bf87-3109f726f91b' to video index 'chatterbox_video'."})
result = await flow.kickoff_async(inputs={'query': "What was the discussed topic for the video stored in video index 'chatterbox_video'?"})

print(f"\n{'='*50}")
print(f"FINAL RESULT")
print(f"{'='*50}")
display(Markdown(result['result']))

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: PixeltableManagerFlow                                                                                    │
│  ID: a29450c2-c623-4403-8ddc-17ed84b1b371                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Flow started with ID: a29450c2-c623-4403-8ddc-17ed84b1b371

Output()

Flow started with query: What was the discussed topic for the video stored in video index 'chatterbox_video'?

--- Using Router Agent to classify query ---

...

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 08dbdf66-5442-4938-9b12-52cab478b532                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The prominent subject in the video labeled as 'chatterbox_video' is about cooking. Precisely,    │
│  it talks about a cooking recipe that includes Mars bars as an ingredient. This recipe appears to be very       │
│  beloved to someone's mum, and there's a reference to a person named Lady Shah.                                 │
│                                                                                                                 │
│  If you require more detailed information, such as the actual process of the recipe or the ingredients needed,  │
│  we can perform more targeted searches based on the specifics you need. If this isn't the topic you were        │
│  hoping to find, please provide more information so we can adjust the search to better suit your needs.         │
│  Lastly, it's suggested that different video categories (like cooking tutorials, interviews, or lectures) be    │
│  assigned unique indexes for systematic organization.                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: PixeltableManagerFlow                                                                                    │
│  ID: a29450c2-c623-4403-8ddc-17ed84b1b371                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


FINAL RESULT


The prominent subject in the video labeled as 'chatterbox_video' is about cooking. Precisely, it talks about a cooking recipe that includes Mars bars as an ingredient. This recipe appears to be very beloved to someone's mum, and there's a reference to a person named Lady Shah. 

If you require more detailed information, such as the actual process of the recipe or the ingredients needed, we can perform more targeted searches based on the specifics you need. If this isn't the topic you were hoping to find, please provide more information so we can adjust the search to better suit your needs. Lastly, it's suggested that different video categories (like cooking tutorials, interviews, or lectures) be assigned unique indexes for systematic organization.

## 📊 Results and Analysis

The output below shows the complete execution trace of the multimodal AI system, including:

- **Router Decision**: How the query was classified
- **Specialist Execution**: The specific actions taken by the chosen agent
- **Tool Interactions**: Direct communication with Pixeltable services
- **Final Synthesis**: The polished user response

### Next Steps:
- Try different query types to see how routing works
- Experiment with different media types (images, audio, documents)
- Monitor the execution flow to understand agent interactions